In [83]:
import pandas as pd
import numpy as np
import os

import geopandas as gpd
import fiona
pd.set_option('display.max_columns', None)

In [88]:
eidl = pd.read_csv('../data/eidl/01 EIDL through 111520.csv', nrows=10)

columns = eidl.columns
columns = columns.to_list()
columns.append('EXTRA1')
columns.append('EXTRA2')
columns.append('EXTRA3')

eidlOne = pd.read_csv('../data/eidl/01 EIDL through 111520.csv', names=columns)
eidlOne = eidlOne.loc[eidlOne['LEGALENTITYSTATECD'] == 'GA']
eidlTwo = pd.read_csv('../data/eidl/02 EIDL through 111520.csv', names=columns)
eidlTwo = eidlTwo.loc[eidlTwo['LEGALENTITYSTATECD'] == 'GA']
eidlThree = pd.read_csv('../data/eidl/03 EIDL through 111520.csv', names=columns)
eidlThree = eidlThree.loc[eidlThree['LEGALENTITYSTATECD'] == 'GA']
eidlFour = pd.read_csv('../data/eidl/03 EIDL through 111520.csv', names=columns)
eidlFour = eidlFour.loc[eidlFour['LEGALENTITYSTATECD'] == 'GA']
eidlFive = pd.read_csv('../data/eidl/03 EIDL through 111520.csv', names=columns)
eidlFive = eidlFive.loc[eidlFive['LEGALENTITYSTATECD'] == 'GA']

dfs = [eidlOne,eidlTwo,eidlThree,eidlFour,eidlFive]
eidl = pd.concat(dfs)
eidl.reset_index(inplace=True, drop=True)

eidl = eidl[['ACTIONDATE','FAIN','AWARDEEORRECIPIENTLEGALENTITYNAME','LEGALENTITYADDRLINE1',
             'LEGALENTITYCITYNAME','LEGALENTITYSTATECD','LEGALENTITYZIP5','LEGALENTITYCONGRESSIONALDISTRICT',
             'FACEVALUEOFDIRECTLOANORLOANGUARANTEE']]

eidl.rename(columns={'ACTIONDATE'                          :'Date',
                     'AWARDEEORRECIPIENTLEGALENTITYNAME'   : 'Recipient',
                     'LEGALENTITYADDRLINE1'                : 'Address',
                     'LEGALENTITYCITYNAME'                 :'City',
                     'LEGALENTITYSTATECD'                  :'State',
                     'LEGALENTITYZIP5'                     :'ZCTA',
                     'LEGALENTITYCONGRESSIONALDISTRICT'    :'Congress',
                     'FACEVALUEOFDIRECTLOANORLOANGUARANTEE':'Amount'}, inplace=True)

eidl['Businesses'] = 1

eidl.Recipient = eidl.Recipient.str.title()
eidl.Address = eidl.Address.str.title()
eidl.City = eidl.City.str.title()

eidl.Recipient = eidl.Recipient.str.strip()
eidl.Address = eidl.Address.str.strip()
eidl.City = eidl.City.str.strip()
eidl.ZCTA = eidl.ZCTA.str.strip()

eidl['Amount'].fillna(0, inplace=True)
eidl['Amount'] = eidl['Amount'].astype(float)
eidl['Amount'] = eidl['Amount'].astype(int)

# merge with correct city and city location
eidlCities = pd.read_excel('../data/eidl/eidlCitiesWalk.xlsx', sheet_name="eidlCities")
cities = pd.read_excel('../data/eidl/eidlCitiesWalk.xlsx', sheet_name="eidlCitiesSuccess", usecols="A,D,E,F")

eidlCities = pd.merge(eidlCities,cities,how='left',on='TrueCity')
eidl = pd.merge(eidl,eidlCities,how='left',on='City')

eidl = eidl[['Date','FAIN','Recipient',
             'Address','TrueCity','County','State','ZCTA','Congress',
             'Amount','Businesses']]

eidl.rename(columns={'Businesses':'Loans',
                     'TrueCity':'City'}, inplace=True)

eidl.drop_duplicates(subset=['FAIN'], inplace=True)
eidl.reset_index(inplace=True, drop=True)

counties = gpd.read_file('../../data/spatial/ga-custom/ga-counties.geojson')
counties = counties[['GEOID','worksource_county','CDRegion','geometry']]
counties.rename(columns={'worksource_county':'County','GEOID':'FIPS'}, inplace=True)
eidl = pd.merge(eidl,counties,how='left',on='County')

msa = pd.read_excel('../../data/geographies/cbsa.xlsx',
                    sheet_name="gaCBSA",
                    usecols="A,D,M",
                    converters={'FIPS_COUNTY':str})

msa.rename(columns={'CBSA Code':'CBSA',
                    'CBSA Title':'MSA',
                    'FIPS_COUNTY':'FIPS'}, inplace=True)

eidl = pd.merge(eidl,msa,how='left',on='FIPS')
eidl['CBSA'].fillna('NA', inplace=True)

eidl = eidl[['Date','FAIN','Recipient',
             'Address','City','FIPS','County','State','ZCTA','Congress','CDRegion','CBSA','MSA',
             'Amount','Loans']]

FileNotFoundError: [Errno 2] File ../data/eidl/01 EIDL through 111520.csv does not exist: '../data/eidl/01 EIDL through 111520.csv'

In [85]:
eidl['MSA'].fillna('None', inplace=True)
eidl['CDRegion'].fillna('None', inplace=True)

In [86]:
eidlTest = eidl.loc[ eidl['MSA'] == 'Rome, GA' ]

In [87]:
eidl['Date'] =  pd.to_datetime(eidl['Date'])

In [24]:
# do rollups by zip and city

In [25]:
# every city and zip needs to be assigned a single county, a single msa, and a single cd region for easy lookups
# get each eidl record a city, county, an msa, and a cd region
# rollup by zip, msa, and cd region
# sort by amount in descending order
# drop duplicate ZCTAs, keping first (the one with most dollars)

In [26]:
eidlZIP = eidl.groupby('ZCTA').agg({'Amount':'sum','Loans':'sum'})
eidlZIP.reset_index(inplace=True)
eidlZIP['ZCTA'] = eidlZIP['ZCTA'].astype(str)
eidlZIP.sort_values(['ZCTA','Amount'], ascending=False, inplace=True)

In [27]:
zcta = pd.read_excel('../data/spatial/zips/Zip_to_zcta_crosswalk_2020.xlsx',
                     usecols="A,E",
                     converters={'ZIP_CODE':str,'ZCTA':str})

eidlZIP = pd.merge(eidlZIP, zcta, how='left', left_on='ZCTA',right_on='ZIP_CODE')

eidlZIP.rename(columns={'ZCTA_y':'ZCTA'}, inplace=True)
eidlZIP = eidlZIP[['ZCTA','Amount','Loans']]
eidlZIP = eidlZIP.groupby('ZCTA').agg({'Amount':'sum','Loans':'sum'}).reset_index()

zctaShape = gpd.read_file('../data/spatial/ga-custom/gaZips.geojson')
eidlZIP = pd.merge(eidlZIP,zctaShape,how='left',on='ZCTA')
eidlZIP = eidlZIP[['ZCTA','County','MSA','CDRegion','Amount','Loans','geometry']]
eidlZIP = gpd.GeoDataFrame(eidlZIP, geometry='geometry')

In [28]:
# now roll by city

eidlCity = eidl.groupby(['City','County','FIPS']).agg({'Amount':'sum','Loans':'sum'}).reset_index()
eidlCity = pd.merge(eidlCity,msa,how='left',on='FIPS')
eidlCity = pd.merge(eidlCity,counties,how='left',on='FIPS')
eidlCity = pd.merge(eidlCity,cities,how='left',left_on='City',right_on='TrueCity')
eidlCity = eidlCity[['City','County','MSA','CDRegion','Amount','Loans','Latitude','Longitude']]
eidlCity.rename(columns={'County_x':'County'}, inplace=True)

eidlCity = gpd.GeoDataFrame(eidlCity, geometry=gpd.points_from_xy(eidlCity.Longitude, eidlCity.Latitude))

eidlCity = eidlCity[['City','County','MSA','CDRegion','Amount','Loans','geometry']]

In [29]:
# now roll up by counties
eidlCounty = eidl.groupby(['County','FIPS']).agg({'Amount':'sum','Loans':'sum'}).reset_index()
eidlCounty = pd.merge(eidlCounty,msa,how='left',on='FIPS')
eidlCounty = pd.merge(eidlCounty,counties,how='left',on='FIPS')
eidlCounty = eidlCounty[['County_x','MSA','CDRegion','Amount','Loans','geometry']]
eidlCounty.rename(columns={'County_x':'County'}, inplace=True)
eidlCounty = gpd.GeoDataFrame(eidlCounty, geometry=eidlCounty['geometry'])

In [30]:
eidlCounty['MSA'].fillna('None', inplace=True)
eidlCounty['CDRegion'].fillna('None', inplace=True)

eidlZIP['MSA'].fillna('None', inplace=True)
eidlZIP['CDRegion'].fillna('None', inplace=True)

eidlCity['MSA'].fillna('None', inplace=True)
eidlCity['CDRegion'].fillna('None', inplace=True)

In [31]:
plus150 = pd.read_csv('./data/pppPlus150/150k plus PPP through 112420.csv')
plus150[['State2','CD']] = plus150['CD'].str.split('-',expand=True)
plus150['CD'] = plus150['State2']+'-'+plus150['CD']
plus150['State'].fillna(plus150['State2'], inplace=True)
plus150 = plus150.loc[plus150['State'] == 'GA']
plus150['Businesses']=1

plus150 = plus150[['DateApproved','BusinessName','BusinessType','NAICSCode','RaceEthnicity','Gender',
                   'Veteran','NonProfit',
                   'Address','City','State','Zip',
                   'JobsReported','LoanAmount','Businesses']]

plus150.rename(columns={'DateApproved':'Date',
                        'BusinessName':'Recipient',
                        'NAICSCode':'NAICS',
                        'Zip':'ZCTA',
                        'LoanAmount':'Amount',
                        'JobsReported':'Jobs'}, inplace=True)

plus150['ZCTA'].fillna(0, inplace=True)
plus150['ZCTA'] = plus150['ZCTA'].astype(int)
plus150['ZCTA'] = plus150['ZCTA'].astype(str)

plus150['Jobs'].fillna(0, inplace=True)
plus150['Jobs'] = plus150['Jobs'].astype(int)

plus150['Amount'].fillna(0, inplace=True)
plus150['Amount'] = plus150['Amount'].astype(int)

plus150['NAICS'].fillna(0, inplace=True)
plus150['NAICS'] = plus150['NAICS'].astype(int)
plus150['NAICS'] = plus150['NAICS'].astype(str)

plus150.reset_index(inplace=True, drop=True)
plus150.to_csv('./data/gaFinal/pppOver150GA.csv', index=False)



In [32]:
sub150One = pd.read_csv('./data/pppSub150/01 PPP sub 150k through 112420.csv')
sub150Two = pd.read_csv('./data/pppSub150/02 PPP sub 150k through 112420.csv')
sub150Three = pd.read_csv('./data/pppSub150/03 PPP sub 150k through 112420.csv')
sub150Four = pd.read_csv('./data/pppSub150/04 PPP sub 150k through 112420.csv')
sub150Five = pd.read_csv('./data/pppSub150/05 PPP sub 150k through 112420.csv')

sub150s = [sub150One, sub150Two, sub150Three, sub150Four, sub150Five]
sub150 = pd.concat(sub150s)
sub150.reset_index(inplace=True, drop=True)

sub150[['State2','CD']] = sub150['CD'].str.split('-',expand=True)
sub150['CD'] = sub150['State2']+'-'+sub150['CD']

sub150['State'].fillna(sub150['State2'], inplace=True)

sub150 = sub150.loc[sub150['State'] == 'GA']

sub150['Businesses'] = 1

sub150 = sub150[['DateApproved','BusinessName','BusinessType','NAICSCode','RaceEthnicity','Gender',
                   'Veteran','NonProfit',
                   'Address','City','State','Zip',
                   'JobsReported','LoanAmount','Businesses']]

sub150.rename(columns={'DateApproved':'Date',
                        'BusinessName':'Recipient',
                        'NAICSCode':'NAICS',
                        'Zip':'ZCTA',
                        'LoanAmount':'Amount',
                        'JobsReported':'Jobs'}, inplace=True)

sub150.rename(columns={'DateApproved':'Date',
                        'BusinessName':'Recipient',
                        'NAICSCode':'NAICS',
                        'Zip':'ZCTA',
                        'LoanAmount':'Amount',
                        'JobsReported':'Jobs'}, inplace=True)

sub150.reset_index(inplace=True, drop=True)

sub150['ZCTA'].fillna(0, inplace=True)
sub150['ZCTA'] = sub150['ZCTA'].astype(int)
sub150['ZCTA'] = sub150['ZCTA'].astype(str)

sub150['Jobs'].fillna(0, inplace=True)
sub150['Jobs'] = sub150['Jobs'].astype(int)

sub150['Amount'].fillna(0, inplace=True)
sub150['Amount'] = sub150['Amount'].astype(int)

sub150['NAICS'].fillna(0, inplace=True)
sub150['NAICS'] = sub150['NAICS'].astype(int)
sub150['NAICS'] = sub150['NAICS'].astype(str)

ppp = pd.concat([plus150, sub150])
ppp.reset_index(inplace=True, drop=True)

ppp.Recipient = ppp.Recipient.str.title()
ppp.Address = ppp.Address.str.title()
ppp.City = ppp.City.str.title()

ppp['BusinessType'] = ppp['BusinessType'].str.replace(' ','')
ppp['BusinessType'] = ppp['BusinessType'].str.replace('(','')
ppp['BusinessType'] = ppp['BusinessType'].str.replace(')','')

ppp.NonProfit.unique()

array([nan, 'Y'], dtype=object)

In [33]:
ppp['BusinessType'] = ppp['BusinessType'].str.replace("LimitedLiabilityCompanyLLC",'LLC')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("Corporation",'Corp')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("Non-ProfitOrganization",'Non-Profit')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("SubchapterSCorp",'Chapter')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("LimitedLiabilityPartnership",'LLP')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("nan",'U')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("ProfessionalAssociation",'Prof. Assoc.')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("SoleProprietorship",'Sole Oroprietor')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("EmployeeStockOwnershipPlanESOP",'ESOP')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("IndependentContractors",'Contractor')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("Self-EmployedIndividuals",'Self-Employed')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("Non-ProfitChildcareCenter",'Childcare')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("JointVenture",'Joint Venture')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("RolloverasBusinessStart-UpsROB",'ROB')
ppp['BusinessType'] = ppp['BusinessType'].str.replace("TeUtinCommon",'Common-Tenant')

In [34]:
ppp['RaceEthnicity'] = ppp['RaceEthnicity'].str.replace("Unanswered",'U')
ppp['Gender'] = ppp['Gender'].str.replace("Unanswered",'U')
ppp['Veteran'] = ppp['Veteran'].str.replace("Unanswered",'U')
ppp['NonProfit'].fillna('N', inplace=True)

In [35]:
naics = pd.read_excel('../data/crosswalks.xlsx',
                      sheet_name='naics',
                      usecols='B,D,I,J',
                      converters={'6-Digit':str})

ppp = pd.merge(ppp,naics,how='left',left_on='NAICS',right_on='6-Digit')

ppp = ppp[['Date','Recipient',
           'BusinessType','NAICS','Industry-2','Industry-3','Industry-6',
           'RaceEthnicity','Gender','Veteran','NonProfit',
           'Address','City','State','ZCTA',
           'Jobs','Amount','Businesses']]

ppp.rename(columns={'Industry-6':'Industry'}, inplace=True)



In [36]:
ppp.drop_duplicates(inplace=True)

In [37]:
# there are some entries we know were incorrectly assigned to Georgia, when in fact they were in another state
ppp = ppp.loc[ppp['City'] != 'Port Charlotte']
ppp = ppp.loc[ppp['City'] != 'Alexandria']
ppp = ppp.loc[ppp['City'] != 'Pinellas Park']
ppp = ppp.loc[ppp['City'] != 'Selma']

pppCitiesTrue = pd.read_excel('./data/pppCitiesWalk.xlsx',
                              sheet_name="pppCitiesSuccess",
                              converters={'FIPS':str,'CBSA':str})

pppCities = pd.read_excel('./data/pppCitiesWalk.xlsx', sheet_name="pppCities")

pppGeo = pd.merge(pppCities, pppCitiesTrue, how='left', on='TrueCity')

pppGeo = pppGeo[['City','TrueCity','ZCTA','FIPS','County','CBSA','MSA','CDRegion']]
ppp = pd.merge(ppp,pppGeo,how='left',on='City')

ppp = ppp[['Date','Recipient','BusinessType','NAICS','Industry-2','Industry-3','Industry',
           'RaceEthnicity','Gender','Veteran','NonProfit',
           'Address','TrueCity','ZCTA_y','FIPS','County','CBSA','MSA','CDRegion',
           'Jobs','Amount','Businesses']]

ppp.rename(columns={'ZCTA_y':'ZCTA',
                    'TrueCity':'City',
                    'Businesses':'Loans'}, inplace=True)

In [38]:
ppp['MSA'].fillna('None', inplace=True)
ppp['CDRegion'].fillna('None', inplace=True)
ppp['Date'] =  pd.to_datetime(ppp['Date'])

In [39]:
pppCity = ppp.groupby(['City','Industry-2']).agg({'Amount':'sum','Loans':'sum','Jobs':'sum'}).reset_index()

In [40]:
pppCity = pd.pivot_table(pppCity, 
                                values=['Amount','Loans','Jobs'],
                                index=['City'],
                                columns=['Industry-2'],
                                aggfunc=np.sum,
                                fill_value=0)

pppCity.columns = list(map("_".join, pppCity.columns))
pppCity.reset_index(inplace=True)

In [41]:
# rollup by city
pppCitiesTrue = pd.read_excel('./data/pppCitiesWalk.xlsx',
                              sheet_name="pppCitiesSuccess",
                              converters={'FIPS':str,'CBSA':str},
                              usecols="A,D,E,H,I,K")

pppCitiesTrue.rename(columns={'TrueCity':'City'}, inplace=True)

pppCity.sort_values('City', inplace=True)

pppCity = pd.merge(pppCity,pppCitiesTrue,how='left',on='City')
pppCity = gpd.GeoDataFrame(pppCity,
                           geometry=gpd.points_from_xy(pppCity.Longitude, pppCity.Latitude))
del pppCity['Latitude']
del pppCity['Longitude']

pppCity.fillna(0, inplace=True)

In [42]:
pppZIP = ppp.groupby(['ZCTA','Industry-2']).agg({'Amount':'sum','Loans':'sum','Jobs':'sum'}).reset_index()
pppZIP = pd.pivot_table(pppZIP, 
                                values=['Amount','Loans','Jobs'],
                                index=['ZCTA'],
                                columns=['Industry-2'],
                                aggfunc=np.sum,
                                fill_value=0)

pppZIP.columns = list(map("_".join, pppZIP.columns))
pppZIP.reset_index(inplace=True)

pppZIP['ZCTA'] = pppZIP['ZCTA'].astype(int)
pppZIP['ZCTA'] = pppZIP['ZCTA'].astype(str)

pppZIP.fillna(0, inplace=True)

zctaShape = gpd.read_file('../data/spatial/ga-custom/gaZips.geojson')
pppZIP = pd.merge(pppZIP,zctaShape,how='left',on='ZCTA')

pppZIP = gpd.GeoDataFrame(pppZIP, geometry='geometry')

In [43]:
pppCounty = ppp.groupby(['County','Industry-2']).agg({'Amount':'sum','Loans':'sum','Jobs':'sum'}).reset_index()
pppCounty = pd.pivot_table(pppCounty, 
                                values=['Amount','Loans','Jobs'],
                                index=['County'],
                                columns=['Industry-2'],
                                aggfunc=np.sum,
                                fill_value=0)

pppCounty.columns = list(map("_".join, pppCounty.columns))
pppCounty.reset_index(inplace=True)

pppCounty.fillna(0, inplace=True)
pppCounty = pd.merge(pppCounty,counties,how='left',on='County')

pppCounty = pd.merge(pppCounty,msa,how='left',on='FIPS')
del pppCounty['CBSA']
del pppCounty['FIPS']



In [46]:
pppCity['MSA'] = pppCity['MSA'].str.replace('0','None')
pppZIP['MSA'] = pppZIP['MSA'].str.replace('0','None')
pppCounty['MSA'] = pppCounty['MSA'].str.replace('0','None')

pppCounty['MSA'].fillna('None', inplace=True)
pppCounty['CDRegion'].fillna('None', inplace=True)

pppZIP['MSA'].fillna('None', inplace=True)
pppZIP['CDRegion'].fillna('None', inplace=True)

pppCity['MSA'].fillna('None', inplace=True)
pppCity['CDRegion'].fillna('None', inplace=True)

In [48]:
pppAmountCols = [  'Amount_Accommodation and Food Services',
                         'Amount_Administrative and Support and Waste Management and Remediation Services',
                         'Amount_Agriculture, Forestry, Fishing and Hunting',
                         'Amount_Arts, Entertainment, and Recreation',
                         'Amount_Construction',
                         'Amount_Educational Services',
                         'Amount_Finance and Insurance',
                         'Amount_Health Care and Social Assistance',
                         'Amount_Information',
                         'Amount_Management of Companies and Enterprises',
                         'Amount_Manufacturing',
                         'Amount_Mining, Quarrying, and Oil and Gas Extraction',
                         'Amount_Other Services (except Public Administration)',
                         'Amount_Professional, Scientific, and Technical Services',
                         'Amount_Public Administration',
                         'Amount_Real Estate and Rental and Leasing',
                         'Amount_Retail Trade',
                         'Amount_Transportation and Warehousing',
                         'Amount_Utilities',
                         'Amount_Wholesale Trade']
pppLoansCols = [  'Loans_Accommodation and Food Services',
                         'Loans_Administrative and Support and Waste Management and Remediation Services',
                         'Loans_Agriculture, Forestry, Fishing and Hunting',
                         'Loans_Arts, Entertainment, and Recreation',
                         'Loans_Construction',
                         'Loans_Educational Services',
                         'Loans_Finance and Insurance',
                         'Loans_Health Care and Social Assistance',
                         'Loans_Information',
                         'Loans_Management of Companies and Enterprises',
                         'Loans_Manufacturing',
                         'Loans_Mining, Quarrying, and Oil and Gas Extraction',
                         'Loans_Other Services (except Public Administration)',
                         'Loans_Professional, Scientific, and Technical Services',
                         'Loans_Public Administration',
                         'Loans_Real Estate and Rental and Leasing',
                         'Loans_Retail Trade',
                         'Loans_Transportation and Warehousing',
                         'Loans_Utilities',
                         'Loans_Wholesale Trade']
pppJobsCols = [  'Jobs_Accommodation and Food Services',
                         'Jobs_Administrative and Support and Waste Management and Remediation Services',
                         'Jobs_Agriculture, Forestry, Fishing and Hunting',
                         'Jobs_Arts, Entertainment, and Recreation',
                         'Jobs_Construction',
                         'Jobs_Educational Services',
                         'Jobs_Finance and Insurance',
                         'Jobs_Health Care and Social Assistance',
                         'Jobs_Information',
                         'Jobs_Management of Companies and Enterprises',
                         'Jobs_Manufacturing',
                         'Jobs_Mining, Quarrying, and Oil and Gas Extraction',
                         'Jobs_Other Services (except Public Administration)',
                         'Jobs_Professional, Scientific, and Technical Services',
                         'Jobs_Public Administration',
                         'Jobs_Real Estate and Rental and Leasing',
                         'Jobs_Retail Trade',
                         'Jobs_Transportation and Warehousing',
                         'Jobs_Utilities']

In [49]:
pppCity['Amount'] = pppCity[ pppAmountCols ].sum(axis=1)
pppCity['Loans'] = pppCity[ pppLoansCols ].sum(axis=1)
pppCity['Jobs'] = pppCity[ pppJobsCols ].sum(axis=1)

In [55]:
pppCounty['Amount'] = pppCounty[ pppAmountCols ].sum(axis=1)
pppCounty['Loans'] = pppCounty[ pppLoansCols ].sum(axis=1)
pppCounty['Jobs'] = pppCounty[ pppJobsCols ].sum(axis=1)

pppZIP['Amount'] = pppZIP[ pppAmountCols ].sum(axis=1)
pppZIP['Loans'] = pppZIP[ pppLoansCols ].sum(axis=1)
pppZIP['Jobs'] = pppZIP[ pppJobsCols ].sum(axis=1)

In [61]:
pppZIP = pppZIP[[  'ZCTA',
                         'County',
                         'MSA',
                         'CDRegion',
                         'Amount_Accommodation and Food Services',
                         'Amount_Administrative and Support and Waste Management and Remediation Services',
                         'Amount_Agriculture, Forestry, Fishing and Hunting',
                         'Amount_Arts, Entertainment, and Recreation',
                         'Amount_Construction',
                         'Amount_Educational Services',
                         'Amount_Finance and Insurance',
                         'Amount_Health Care and Social Assistance',
                         'Amount_Information',
                         'Amount_Management of Companies and Enterprises',
                         'Amount_Manufacturing',
                         'Amount_Mining, Quarrying, and Oil and Gas Extraction',
                         'Amount_Other Services (except Public Administration)',
                         'Amount_Professional, Scientific, and Technical Services',
                         'Amount_Public Administration',
                         'Amount_Real Estate and Rental and Leasing',
                         'Amount_Retail Trade',
                         'Amount_Transportation and Warehousing',
                         'Amount_Utilities',
                         'Amount_Wholesale Trade',
                         'Amount',
                         'Jobs_Accommodation and Food Services',
                         'Jobs_Administrative and Support and Waste Management and Remediation Services',
                         'Jobs_Agriculture, Forestry, Fishing and Hunting',
                         'Jobs_Arts, Entertainment, and Recreation',
                         'Jobs_Construction',
                         'Jobs_Educational Services',
                         'Jobs_Finance and Insurance',
                         'Jobs_Health Care and Social Assistance',
                         'Jobs_Information',
                         'Jobs_Management of Companies and Enterprises',
                         'Jobs_Manufacturing',
                         'Jobs_Mining, Quarrying, and Oil and Gas Extraction',
                         'Jobs_Other Services (except Public Administration)',
                         'Jobs_Professional, Scientific, and Technical Services',
                         'Jobs_Public Administration',
                         'Jobs_Real Estate and Rental and Leasing',
                         'Jobs_Retail Trade',
                         'Jobs_Transportation and Warehousing',
                         'Jobs_Utilities',
                         'Jobs_Wholesale Trade',
                         'Jobs',
                         'Loans_Accommodation and Food Services',
                         'Loans_Administrative and Support and Waste Management and Remediation Services',
                         'Loans_Agriculture, Forestry, Fishing and Hunting',
                         'Loans_Arts, Entertainment, and Recreation',
                         'Loans_Construction',
                         'Loans_Educational Services',
                         'Loans_Finance and Insurance',
                         'Loans_Health Care and Social Assistance',
                         'Loans_Information',
                         'Loans_Management of Companies and Enterprises',
                         'Loans_Manufacturing',
                         'Loans_Mining, Quarrying, and Oil and Gas Extraction',
                         'Loans_Other Services (except Public Administration)',
                         'Loans_Professional, Scientific, and Technical Services',
                         'Loans_Public Administration',
                         'Loans_Real Estate and Rental and Leasing',
                         'Loans_Retail Trade',
                         'Loans_Transportation and Warehousing',
                         'Loans_Utilities',
                         'Loans_Wholesale Trade',
                         'Loans',
                         'geometry']]

pppZIP = gpd.GeoDataFrame(pppZIP, geometry=pppZIP['geometry'])

In [57]:
pppCounty = pppCounty[[  'County',
                         'MSA',
                         'CDRegion',
                         'Amount_Accommodation and Food Services',
                         'Amount_Administrative and Support and Waste Management and Remediation Services',
                         'Amount_Agriculture, Forestry, Fishing and Hunting',
                         'Amount_Arts, Entertainment, and Recreation',
                         'Amount_Construction',
                         'Amount_Educational Services',
                         'Amount_Finance and Insurance',
                         'Amount_Health Care and Social Assistance',
                         'Amount_Information',
                         'Amount_Management of Companies and Enterprises',
                         'Amount_Manufacturing',
                         'Amount_Mining, Quarrying, and Oil and Gas Extraction',
                         'Amount_Other Services (except Public Administration)',
                         'Amount_Professional, Scientific, and Technical Services',
                         'Amount_Public Administration',
                         'Amount_Real Estate and Rental and Leasing',
                         'Amount_Retail Trade',
                         'Amount_Transportation and Warehousing',
                         'Amount_Utilities',
                         'Amount_Wholesale Trade',
                         'Amount',
                         'Jobs_Accommodation and Food Services',
                         'Jobs_Administrative and Support and Waste Management and Remediation Services',
                         'Jobs_Agriculture, Forestry, Fishing and Hunting',
                         'Jobs_Arts, Entertainment, and Recreation',
                         'Jobs_Construction',
                         'Jobs_Educational Services',
                         'Jobs_Finance and Insurance',
                         'Jobs_Health Care and Social Assistance',
                         'Jobs_Information',
                         'Jobs_Management of Companies and Enterprises',
                         'Jobs_Manufacturing',
                         'Jobs_Mining, Quarrying, and Oil and Gas Extraction',
                         'Jobs_Other Services (except Public Administration)',
                         'Jobs_Professional, Scientific, and Technical Services',
                         'Jobs_Public Administration',
                         'Jobs_Real Estate and Rental and Leasing',
                         'Jobs_Retail Trade',
                         'Jobs_Transportation and Warehousing',
                         'Jobs_Utilities',
                         'Jobs_Wholesale Trade',
                         'Jobs',
                         'Loans_Accommodation and Food Services',
                         'Loans_Administrative and Support and Waste Management and Remediation Services',
                         'Loans_Agriculture, Forestry, Fishing and Hunting',
                         'Loans_Arts, Entertainment, and Recreation',
                         'Loans_Construction',
                         'Loans_Educational Services',
                         'Loans_Finance and Insurance',
                         'Loans_Health Care and Social Assistance',
                         'Loans_Information',
                         'Loans_Management of Companies and Enterprises',
                         'Loans_Manufacturing',
                         'Loans_Mining, Quarrying, and Oil and Gas Extraction',
                         'Loans_Other Services (except Public Administration)',
                         'Loans_Professional, Scientific, and Technical Services',
                         'Loans_Public Administration',
                         'Loans_Real Estate and Rental and Leasing',
                         'Loans_Retail Trade',
                         'Loans_Transportation and Warehousing',
                         'Loans_Utilities',
                         'Loans_Wholesale Trade',
                         'Loans',
                         'geometry']]

pppCounty = gpd.GeoDataFrame(pppCounty, geometry=pppCounty['geometry'])

In [53]:
pppCity = pppCity[[  'City',
                         'County',
                         'MSA',
                         'CDRegion',
                         'Amount_Accommodation and Food Services',
                         'Amount_Administrative and Support and Waste Management and Remediation Services',
                         'Amount_Agriculture, Forestry, Fishing and Hunting',
                         'Amount_Arts, Entertainment, and Recreation',
                         'Amount_Construction',
                         'Amount_Educational Services',
                         'Amount_Finance and Insurance',
                         'Amount_Health Care and Social Assistance',
                         'Amount_Information',
                         'Amount_Management of Companies and Enterprises',
                         'Amount_Manufacturing',
                         'Amount_Mining, Quarrying, and Oil and Gas Extraction',
                         'Amount_Other Services (except Public Administration)',
                         'Amount_Professional, Scientific, and Technical Services',
                         'Amount_Public Administration',
                         'Amount_Real Estate and Rental and Leasing',
                         'Amount_Retail Trade',
                         'Amount_Transportation and Warehousing',
                         'Amount_Utilities',
                         'Amount_Wholesale Trade',
                         'Amount',
                         'Jobs_Accommodation and Food Services',
                         'Jobs_Administrative and Support and Waste Management and Remediation Services',
                         'Jobs_Agriculture, Forestry, Fishing and Hunting',
                         'Jobs_Arts, Entertainment, and Recreation',
                         'Jobs_Construction',
                         'Jobs_Educational Services',
                         'Jobs_Finance and Insurance',
                         'Jobs_Health Care and Social Assistance',
                         'Jobs_Information',
                         'Jobs_Management of Companies and Enterprises',
                         'Jobs_Manufacturing',
                         'Jobs_Mining, Quarrying, and Oil and Gas Extraction',
                         'Jobs_Other Services (except Public Administration)',
                         'Jobs_Professional, Scientific, and Technical Services',
                         'Jobs_Public Administration',
                         'Jobs_Real Estate and Rental and Leasing',
                         'Jobs_Retail Trade',
                         'Jobs_Transportation and Warehousing',
                         'Jobs_Utilities',
                         'Jobs_Wholesale Trade',
                         'Jobs',
                         'Loans_Accommodation and Food Services',
                         'Loans_Administrative and Support and Waste Management and Remediation Services',
                         'Loans_Agriculture, Forestry, Fishing and Hunting',
                         'Loans_Arts, Entertainment, and Recreation',
                         'Loans_Construction',
                         'Loans_Educational Services',
                         'Loans_Finance and Insurance',
                         'Loans_Health Care and Social Assistance',
                         'Loans_Information',
                         'Loans_Management of Companies and Enterprises',
                         'Loans_Manufacturing',
                         'Loans_Mining, Quarrying, and Oil and Gas Extraction',
                         'Loans_Other Services (except Public Administration)',
                         'Loans_Professional, Scientific, and Technical Services',
                         'Loans_Public Administration',
                         'Loans_Real Estate and Rental and Leasing',
                         'Loans_Retail Trade',
                         'Loans_Transportation and Warehousing',
                         'Loans_Utilities',
                         'Loans_Wholesale Trade',
                         'Loans',
                         'geometry']]

pppCity = gpd.GeoDataFrame(pppCity, geometry=pppCity['geometry'])

In [62]:
# write out all files
eidl.to_csv('./data/gaFinal/eidlGA.csv', index=False)
eidlCity.to_file('./data/gaFinal/spatialFinal/eidlCityRoll.geojson', driver="GeoJSON")
eidlCounty.to_file('./data/gaFinal/spatialFinal/eidlCountyRoll.geojson', driver="GeoJSON")
eidlZIP.to_file('./data/gaFinal/spatialFinal/eidlZipRoll.geojson', driver="GeoJSON")
ppp.to_csv('./data/gaFinal/pppGA.csv', index=False)
pppCity.to_file('./data/gaFinal/spatialFinal/pppCityRoll.geojson', driver="GeoJSON")
pppZIP.to_file('./data/gaFinal/spatialFinal/pppZipRoll.geojson', driver="GeoJSON")
pppCounty.to_file('./data/gaFinal/spatialFinal/pppCountyRoll.geojson', driver="GeoJSON")

In [28]:
# ideally you would create api access to this, but you don't have the capacity
# instead, break the eidl and ppp files into sub folders by geography and industry
# when a user selects data, the map will visualize statewide since its compressed mbtiles
# a table will not be built though, until a geography is selected
# once this selection happend, the function it fires will build a url to the corresponding github folder
# with the right subset of the data based on the geography selection

# slice files for individual MSAs (counties without an MSA will get their own sub folder too)
# slice files for individual CD regions
# if a user just selects an individual county then its whole MSA folder will be pulled in

In [63]:
eidlMsa = eidl['MSA'].to_list()
eidlMsa = list( set( eidlMsa ) )

for msa in eidlMsa:
    msaDir = msa.replace(" ", "")
    data = eidl.loc[ eidl['MSA'] == msa ]
    data.to_csv('./data/gaFinal/eidlSub/eidl'+msaDir+'.csv', index=False)
    
eidlCD = eidl['CDRegion'].to_list()
eidlCD = list( set( eidlCD ) )

for cd in eidlCD:
    cdDir = cd.replace(" ", "")
    data = eidl.loc[ eidl['CDRegion'] == cd ]
    data.to_csv('./data/gaFinal/eidlSub/eidl'+cdDir+'.csv', index=False)

In [64]:
pppMsa = ppp['MSA'].to_list()
pppMsa = list( set( pppMsa ) )

for msa in pppMsa:
    msaDir = msa.replace(" ", "")
    data = ppp.loc[ ppp['MSA'] == msa ]
    data.to_csv('./data/gaFinal/pppSub/ppp'+msaDir+'.csv', index=False)
    

pppCD = ppp['CDRegion'].to_list()
pppCD = list( set( pppCD ) )

for cd in pppCD:
    cdDir = cd.replace(" ", "")
    data = ppp.loc[ ppp['CDRegion'] == cd ]
    data.to_csv('./data/gaFinal/pppSub/ppp'+cdDir+'.csv', index=False)

In [73]:
naics.sort_values('Industry-2', inplace=True)

In [74]:
industries = naics['Industry-2'].unique()
industries = industries.tolist()
for industry in industries:
    print( industry )

Accommodation and Food Services
Administrative and Support and Waste Management and Remediation Services
Agriculture, Forestry, Fishing and Hunting
Arts, Entertainment, and Recreation
Construction
Educational Services
Finance and Insurance
Health Care and Social Assistance
Information
Management of Companies and Enterprises
Manufacturing
Mining, Quarrying, and Oil and Gas Extraction
Other Services (except Public Administration)
Professional, Scientific, and Technical Services
Public Administration
Real Estate and Rental and Leasing
Retail Trade
Transportation and Warehousing
Utilities
Wholesale Trade


In [66]:
naics.head()

,Industry-2,Industry-3,6-Digit,Industry-6
0,"Agriculture, Forestry, Fishing and Hunting",Crop Production,111110,Soybean Farming
1,"Agriculture, Forestry, Fishing and Hunting",Crop Production,111120,Oilseed (except Soybean) Farming
2,"Agriculture, Forestry, Fishing and Hunting",Crop Production,111130,Dry Pea and Bean Farming
3,"Agriculture, Forestry, Fishing and Hunting",Crop Production,111140,Wheat Farming
4,"Agriculture, Forestry, Fishing and Hunting",Crop Production,111150,Corn Farming


In [78]:
pppCity.head()

,City,County,MSA,CDRegion,Amount_Accommodation and Food Services,Amount_Administrative and Support and Waste Management and Remediation Services,"Amount_Agriculture, Forestry, Fishing and Hunting","Amount_Arts, Entertainment, and Recreation",Amount_Construction,Amount_Educational Services,Amount_Finance and Insurance,Amount_Health Care and Social Assistance,Amount_Information,Amount_Management of Companies and Enterprises,Amount_Manufacturing,"Amount_Mining, Quarrying, and Oil and Gas Extraction",Amount_Other Services (except Public Administration),"Amount_Professional, Scientific, and Technical Services",Amount_Public Administration,Amount_Real Estate and Rental and Leasing,Amount_Retail Trade,Amount_Transportation and Warehousing,Amount_Utilities,Amount_Wholesale Trade,Amount,Jobs_Accommodation and Food Services,Jobs_Administrative and Support and Waste Management and Remediation Services,"Jobs_Agriculture, Forestry, Fishing and Hunting","Jobs_Arts, Entertainment, and Recreation",Jobs_Construction,Jobs_Educational Services,Jobs_Finance and Insurance,Jobs_Health Care and Social Assistance,Jobs_Information,Jobs_Management of Companies and Enterprises,Jobs_Manufacturing,"Jobs_Mining, Quarrying, and Oil and Gas Extraction",Jobs_Other Services (except Public Administration),"Jobs_Professional, Scientific, and Technical Services",Jobs_Public Administration,Jobs_Real Estate and Rental and Leasing,Jobs_Retail Trade,Jobs_Transportation and Warehousing,Jobs_Utilities,Jobs_Wholesale Trade,Jobs,Loans_Accommodation and Food Services,Loans_Administrative and Support and Waste Management and Remediation Services,"Loans_Agriculture, Forestry, Fishing and Hunting","Loans_Arts, Entertainment, and Recreation",Loans_Construction,Loans_Educational Services,Loans_Finance and Insurance,Loans_Health Care and Social Assistance,Loans_Information,Loans_Management of Companies and Enterprises,Loans_Manufacturing,"Loans_Mining, Quarrying, and Oil and Gas Extraction",Loans_Other Services (except Public Administration),"Loans_Professional, Scientific, and Technical Services",Loans_Public Administration,Loans_Real Estate and Rental and Leasing,Loans_Retail Trade,Loans_Transportation and Warehousing,Loans_Utilities,Loans_Wholesale Trade,Loans,geometry
0,Abbeville,Wilcox County,None,Central,0,0,147570,0,29022,0,16250,461282,0,0,28922,0,53964,27373,0,0,42564,207662,0,8532,1023141,0,0,28,0,6,0,1,110,0,0,7,0,50,3,0,0,8,22,0,1,235,0,0,6,0,2,0,1,3,0,0,1,0,4,1,0,0,4,4,0,1,27,POINT (-83.37252 31.97705)
1,Acworth,Cobb County,"Atlanta-Sandy Springs-Alpharetta, GA",Metro West,6933663,3368388,213577,969016,10209681,2293075,1488643,8005034,1015306,215672,13090698,353345,6573268,7395460,73742,793990,9613222,3569337,2535000,3438407,82148524,1450,363,33,113,809,180,134,1003,137,37,874,1,686,521,4,109,1023,275,135,226,7887,74,114,2,27,160,26,43,98,18,6,53,2,168,156,4,50,132,77,1,42,1253,POINT (-84.70735 34.03451)
2,Adairsville,Bartow County,"Atlanta-Sandy Springs-Alpharetta, GA",Northwest,2770686,29900,13200,92982,1532002,39100,38020,1645685,5625,0,6468353,0,300167,492103,0,0,1504883,129485,0,2367364,17429555,400,9,3,33,164,10,8,143,0,0,515,0,23,44,0,0,178,15,0,142,1545,13,2,2,4,23,1,4,12,1,0,18,0,16,15,0,0,20,8,0,5,144,POINT (-84.93411 34.36870)
3,Adel,Cook County,None,South,2248363,228911,442033,43969,1463053,122927,162022,556391,90538,0,3056429,0,361157,184034,12500,331608,1042910,126853,0,56833,10530531,621,5,72,34,180,32,25,89,28,0,211,0,62,26,1,24,196,10,0,5,1616,15,5,6,3,16,1,9,11,1,0,15,0,18,8,1,6,25,3,0,3,146,POINT (-83.40714 31.15174)
4,Adrian,Emanuel County,None,Central,31521,72274,35320,3752,103223,0,8668,0,0,0,0,0,89259,0,0,0,0,12200,0,0,356217,10,10,32,1,13,0,1,0,0,0,0,0,17,0,0,0,0,1,0,0,85,1,3,3,1,2,0,1,0,0,0,0,0,4,0,0,0,0,1,0,0,16,POINT (-82.54031 32.56887)


In [82]:
pppCity['Loans_Accommodation and Food Services'].quantile([0.05,.1, .2,.3, .4,.5,.6,.7,.8,.9, .95])

0.05     0.0
0.10     0.0
0.20     0.0
0.30     0.0
0.40     1.0
0.50     2.0
0.60     4.0
0.70     8.0
0.80    17.0
0.90    41.0
0.95    87.0
Name: Loans_Accommodation and Food Services, dtype: float64

In [ ]:
pppZIP = pppZIP.loc[pppZIP['Industry-2'] =="Accommodation and Food Services"]

In [81]:
pppZIP['Amount_Accommodation and Food Services'].quantile([0.05, .2, .4,.6,.8, .95])

0.05          0.0
0.20          0.0
0.40      21258.6
0.60     165642.8
0.80    1478549.2
0.95    7802437.7
Name: Amount_Accommodation and Food Services, dtype: float64